# SQL. Итоговый проект.

**Цель исследования:**  анализ базы данных крупного сервиса для чтения книг по подписке с целью определения предпочтений пользователей, успешных книг, пользовательских обзоров, а также выявления популярных авторов и издательств для разработки ценностного предложения нового продукта.

# Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Доступ к базе данных

In [52]:
# pip install psycopg2

In [53]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Обзор данных

Выведем первые строки каждой таблицы.

**Таблица `books`**

In [54]:
query =('''
    
     SELECT *
     FROM books
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


**Таблица `authors`**

In [55]:
query =('''
    
     SELECT *
     FROM authors
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt


**Таблица `publishers`**

In [56]:
query =('''
    
     SELECT *
     FROM publishers
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace


**Таблица `ratings`**

In [57]:
query =('''
    
     SELECT *
     FROM ratings
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


**Таблица `reviews`**

In [58]:
query =('''
    
     SELECT *
     FROM reviews
     LIMIT 1

''')

pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


# Задания

## Задание 1.

**`Посчитайте, сколько книг вышло после 1 января 2000 года`**

In [59]:
query =('''
    
     SELECT COUNT(book_id)
     FROM books
     WHERE publication_date > '2000-01-01'

''')

pd.io.sql.read_sql(query, con = engine) 

,count
0,819


**`Вывод:`** после 1 января 2000 года вышло **819** книг.

## Задание 2.

**`Для каждой книги посчитайте количество обзоров и среднюю оценку`**

In [60]:
query =('''
    
     SELECT b.title,
            b.book_id,
            ROUND(AVG(ra.rating), 2) AS avg_rating,
            COUNT(DISTINCT re.review_id) AS total_reviews
     FROM books AS b
     INNER JOIN ratings AS ra ON ra.book_id = b.book_id
     LEFT JOIN reviews  AS re ON re.book_id = b.book_id
     GROUP BY b.title,
              b.book_id
     ORDER BY total_reviews DESC;

''')

pd.io.sql.read_sql(query, con = engine) 

,title,book_id,avg_rating,total_reviews
0,Twilight (Twilight #1),948,3.66,7
1,Water for Elephants,963,3.98,6
2,The Glass Castle,734,4.21,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.41,6
4,The Curious Incident of the Dog in the Night-Time,695,4.08,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.67,0
996,The Natural Way to Draw,808,3.00,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.00,0
998,Essential Tales and Poems,221,4.00,0


**`Вывод:`** Мы посчитали количество обзоров и среднюю оценку каждой книги.

## Задание 3.

**`Определите издательство, которое выпустило наибольшее число книг толще 50 страниц`**

In [61]:
query =('''
    
     SELECT p.publisher,
            COUNT(b.book_id) AS books
     FROM publishers AS p
     JOIN books AS b ON p.publisher_id = b.publisher_id
     WHERE b.num_pages > 50
     GROUP BY p.publisher_id  
     ORDER BY books DESC
     LIMIT 1;
     
''')

pd.io.sql.read_sql(query, con = engine) 

,publisher,books
0,Penguin Books,42


**`Вывод:`** Издательство **Penguin Books** выпустило наибольшее число книг толще 50 страниц - **42 книги**.

## Задание 4.

**`Определите автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками`**

In [62]:
query =('''
   SELECT author,
          ROUND (AVG(avg_rating), 2) AS average_rating
   FROM (SELECT author_id,
                title,
                AVG(rating) AS avg_rating
        FROM books AS b
        LEFT JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY title, author_id
        HAVING COUNT(rating_id) >= 50) as query
    LEFT JOIN authors AS a ON query.author_id = a.author_id
    GROUP BY author
    ORDER BY average_rating DESC
    LIMIT 1
''')   
pd.io.sql.read_sql(query, con = engine)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.28


**`Вывод:`**  Автор с самой высокой средней оценкой книг с 50 и более оценками **J.K. Rowling/Mary GrandPré**.

## Задание 5.

**`Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок`**

In [63]:
query =('''

   SELECT ROUND(AVG(subquery.total_reviews), 2) AS avg_review_count
   FROM (SELECT username,
                COUNT(review_id) AS total_reviews
         FROM reviews 
         WHERE username IN (SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 48)
         GROUP BY username) AS subquery
       

''')

pd.io.sql.read_sql(query, con = engine)

,avg_review_count
0,24.0


**`Вывод:`**  Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - **24**.

# Общие выводы

По результатам проведенного анализа  базы данных крупного сервиса для чтения книг по подписке, можно сделать следующие выводы:
- После 1 января 2000 года вышло **819** книг.
- Издательство **Penguin Books** выпустило наибольшее число книг толще 50 страниц - **42 книги**.
- Автор с самой высокой средней оценкой книг с 50 и более оценками **J.K. Rowling/Mary GrandPré**.
- Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - **24**.